In [ ]:
from time import perf_counter

from transformers import AutoTokenizer
import torch

import pandas as pd
from tqdm import tqdm

from models.prefix_tuning.gpt2 import GPT2PrefixTuningForCausalLM
from models.prefix_tuning.gptj import GPTJPrefixTuningForCausalLM

In [ ]:
model_path = "/opt/ml/final-project-level3-nlp-08/outputs/checkpoint-500"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2PrefixTuningForCausalLM.from_pretrained(model_path).to(device)
model.eval()

In [ ]:
@torch.no_grad()
def genreate_from_input(input_text: str, max_length: int = 64, top_k: int = 30, top_p: float = 0.95):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
    )
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    return generated_text, spent

In [ ]:
# input_captions = pd.read_csv("/opt/ml/data/inputs_for_generation/show_attend_and_tell_output_captions.csv")
# input_captions = input_captions.drop(
#     input_captions[input_captions["Unnamed: 0"].apply(lambda x: "<unk>" in x)].index
# )

# top_p = 0.99

# outputs = {"input": [], "gen1": [], "gen2": [], "gen3": [], "gen4": [], "gen5": [], "time_spent": []}
# for input_text in tqdm(input_captions["Unnamed: 0"].sample(100, random_state=42)):
#     outputs["input"].append(input_text := input_text[:-4])
#     avg_t = 0
#     for n in range(1, 6):
#         generated, t = genreate_from_input(input_text, top_p=top_p)
#         outputs[f"gen{n}"].append(generated)
#         avg_t += t
#     outputs["time_spent"].append(avg_t / 5)

# outputs_df = pd.DataFrame(outputs)
# outputs_df.to_csv(f"/opt/ml/analysis/outputs_from_show_attend_and_tell_captions_poem_p0{int(top_p * 100)}.csv", index=False)
# outputs_df

In [ ]:
input_text = "차가 도로를 지나고 있다."
generated_text, spent = genreate_from_input(input_text)

print("generated text:", generated_text, sep="\n")
print(f"\ntime spent: {spent:.2f} sec")

In [ ]:
# model.push_to_hub("prefix-gpt-trinity", private=True)
# tokenizer.push_to_hub("prefix-gpt-trinity", private=True)